In [2]:
import base64
import json
import pandas as pd
import numpy as np
import requests
import tensorflow as tf

In [3]:
df = pd.read_csv("data/Cleaned_HR_Data_Analysis.csv")

In [4]:
from pprint import PrettyPrinter
pp = PrettyPrinter()
pp.pprint(requests.get("https://final-submission-mlops-production.up.railway.app/v1/models/employee-performance-model"))

<Response [200]>


In [12]:
import json
import base64
import requests
import tensorflow as tf

# Data fitur
CATEGORICAL_FEATURES = {
    "BusinessUnit": 10,
    "EmployeeStatus": 2,
    "EmployeeType": 3,
    "PayZone": 3,
    "EmployeeClassificationType": 3,
    "DepartmentType": 6,
    "GenderCode": 2,
    "RaceDesc": 5,
    "MaritalDesc": 4,
    "TrainingType": 2,
    "TrainingOutcome": 4
}

NUMERICAL_FEATURES = [
    "CurrentEmployeeRating",
    "EngagementScore",
    "SatisfactionScore",
    "Work-LifeBalanceScore",
    "TrainingDurationDays",
    "TrainingCost",
    "Age"
]

LABEL_KEY = "PerformanceScore"  # Label yang ingin diprediksi

# Fungsi untuk menyiapkan data dalam format JSON
def prepare_json(inputs: dict):
    feature_spec = {}

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

# Mengambil data dari dataframe (misalnya index ke-53)
inputs = df.loc[53, list(CATEGORICAL_FEATURES.keys()) + NUMERICAL_FEATURES].to_dict()
json_data = prepare_json(inputs)

# Gantilah endpoint dengan yang sesuai untuk model PerformanceScore
endpoint = "https://final-submission-mlops-production.up.railway.app/v1/models/employee-performance-model:predict"
response = requests.post(endpoint, data=json_data)

# Mengambil hasil prediksi
prediction = response.json().get("predictions")

if prediction:
    prediction_probs = prediction[0]  # Output probabilitas untuk setiap kelas (misal: [0.1, 0.3, 0.4, 0.2])
    predicted_index = prediction_probs.index(max(prediction_probs))  # Pilih kelas dengan probabilitas tertinggi

    # Mapping indeks ke label kelas
    label_mapping = {
        0: "PIP",
        1: "Fully Meets",
        2: "Exceeds",
        3: "Needs Improvement"
    }

    result = label_mapping.get(predicted_index, "Unknown")
else:
    result = "Error: No predictions found in response."

print(result)


Error: No predictions found in response.


In [ ]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://final-submission-mlops-production.up.railway.app/v1/models/employee-performance-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "No diabetes" if prediction_value < 0.5 else "Diabetes"
else:
    result = "Error: No predictions found in response."

print(result)

None


In [7]:
print(json_data)

{"signature_name": "serving_default", "instances": [{"examples": {"b64": "Cv8DCh0KFFRyYWluaW5nRHVyYXRpb25EYXlzEgUaAwoBAwoYCg9FbmdhZ2VtZW50U2NvcmUSBRoDCgEDChYKCkdlbmRlckNvZGUSCAoGCgRNYWxlChUKCFJhY2VEZXNjEgkKBwoFV2hpdGUKHgoVV29yay1MaWZlQmFsYW5jZVNjb3JlEgUaAwoBBQoeChVDdXJyZW50RW1wbG95ZWVSYXRpbmcSBRoDCgEFCisKGkVtcGxveWVlQ2xhc3NpZmljYXRpb25UeXBlEg0KCwoJRnVsbC1UaW1lChcKDEJ1c2luZXNzVW5pdBIHCgUKA05FTAoaCgtNYXJpdGFsRGVzYxILCgkKB1dpZG93ZWQKGAoMVHJhaW5pbmdDb3N0EggSBgoErkfdQwoMCgNBZ2USBRoDCgEwChUKB1BheVpvbmUSCgoICgZab25lIEIKHAoORW1wbG95ZWVTdGF0dXMSCgoICgZBY3RpdmUKHAoMVHJhaW5pbmdUeXBlEgwKCgoISW50ZXJuYWwKGwoORGVwYXJ0bWVudFR5cGUSCQoHCgVTYWxlcwocCgxFbXBsb3llZVR5cGUSDAoKCghDb250cmFjdAohCg9UcmFpbmluZ091dGNvbWUSDgoMCgpJbmNvbXBsZXRlChoKEVNhdGlzZmFjdGlvblNjb3JlEgUaAwoBBA=="}}]}


In [8]:
print(response.status_code, response.text)

404 {"status":"error","code":404,"message":"Application not found","request_id":"TrlVXRVkQtKnq4y42AUYfA_4108239275"}


In [9]:
print(response.json())

{'status': 'error', 'code': 404, 'message': 'Application not found', 'request_id': 'TrlVXRVkQtKnq4y42AUYfA_4108239275'}


In [ ]:
import json
import base64
import requests
import tensorflow as tf
import pandas as pd  # Pastikan df sudah didefinisikan

# Data fitur
CATEGORICAL_FEATURES = {
    "BusinessUnit": 10,
    "EmployeeStatus": 2,
    "EmployeeType": 3,
    "PayZone": 3,
    "EmployeeClassificationType": 3,
    "DepartmentType": 6,
    "GenderCode": 2,
    "RaceDesc": 5,
    "MaritalDesc": 4,
    "TrainingType": 2,
    "TrainingOutcome": 4
}

NUMERICAL_FEATURES = [
    "CurrentEmployeeRating",
    "EngagementScore",
    "SatisfactionScore",
    "Work-LifeBalanceScore",
    "TrainingDurationDays",
    "TrainingCost",
    "Age"
]

LABEL_KEY = "PerformanceScore"  # Label yang ingin diprediksi
AVAILABLE_FEATURES = list(CATEGORICAL_FEATURES.keys()) + NUMERICAL_FEATURES  # Gabungkan semua fitur yang valid

def prepare_json(inputs: dict):
    """Menyiapkan payload JSON untuk dikirim ke API TensorFlow Serving."""
    filtered_inputs = {k: v for k, v in inputs.items() if k in AVAILABLE_FEATURES}

    feature_spec = {}
    for key, value in filtered_inputs.items():
        if isinstance(value, float):
            feature_spec[key] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
        elif isinstance(value, int):
            feature_spec[key] = tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
        elif isinstance(value, str):
            feature_spec[key] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))
        else:
            feature_spec[key] = tf.train.Feature()  # Handle missing or invalid types

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    json_data = json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    }, default=str)

    print("\n=== JSON Payload Sent ===")
    print(json_data)  # Cek format JSON sebelum dikirim

    return json_data

# Pastikan df sudah ada
try:
    df = pd.read_csv("./data/Cleaned_HR_Data_Analysis.csv")  # Ubah ini sesuai sumber data
    inputs = df.loc[53, AVAILABLE_FEATURES].to_dict()
except Exception as e:
    print("\n=== Error Reading DataFrame ===")
    print(e)
    inputs = {}

# Jika inputs kosong, hentikan eksekusi
if not inputs:
    print("\n=== Error: No valid input data ===")
    exit()

json_data = prepare_json(inputs)

# API endpoint
endpoint = "https://final-submission-mlops-production.up.railway.app/v1/models/employee-performance-model:predict"

# Kirim request ke API
try:
    response = requests.post(endpoint, json=json.loads(json_data), headers={"Content-Type": "application/json"})
    
    print("\n=== API Response Status ===")
    print(response.status_code)

    if response.status_code != 200:
        print("\n=== API Response Error ===")
        print(response.text)  # Print error message dari server
        exit()
    
    # Parse JSON response
    response_json = response.json()
    print("\n=== API Response JSON ===")
    print(response_json)
    
    # Ambil hasil prediksi
    prediction = response_json.get("predictions", [])

    if prediction:
        prediction_probs = prediction[0]  # Output probabilitas untuk setiap kelas
        predicted_index = prediction_probs.index(max(prediction_probs))  # Pilih kelas dengan probabilitas tertinggi

        label_mapping = {0: "Poor", 1: "Average", 2: "Good", 3: "Excellent"}
        result = label_mapping.get(predicted_index, "Unknown")
    else:
        result = "Error: No predictions found in response."
    
    print("\n=== Predicted Result ===")
    print(result)

except requests.exceptions.RequestException as e:
    print("\n=== API Request Error ===")
    print(e)



=== Error Reading DataFrame ===
[Errno 2] No such file or directory: 'employee_data.csv'

=== Error: No valid input data ===

=== JSON Payload Sent ===
{"signature_name": "serving_default", "instances": [{"examples": {"b64": "CgA="}}]}

=== API Response Status ===
400

=== API Response Error ===
{
    "error": "Name: <unknown>, Feature: Age (data type: int64) is required but could not be found.\n\t [[{{function_node __inference_serve_tf_examples_fn_1008875}}{{node ParseExample/ParseExampleV2}}]]"
}

=== API Response JSON ===
{'error': 'Name: <unknown>, Feature: Age (data type: int64) is required but could not be found.\n\t [[{{function_node __inference_serve_tf_examples_fn_1008875}}{{node ParseExample/ParseExampleV2}}]]'}

=== Predicted Result ===
Error: No predictions found in response.


: 

In [17]:
import tensorflow as tf

model = tf.saved_model.load("./output/serving_model/1742256020")
print(model.signatures["serving_default"])


ConcreteFunction signature_wrapper(*, examples)
  Args:
    examples: string Tensor, shape=(None,)
  Returns:
    {'outputs': <1>}
      <1>: float32 Tensor, shape=(None, 4)


In [ ]:
"""
0 (PIP) → Kinerja buruk, butuh perbaikan segera.
1 (Fully Meets) → Memenuhi harapan pekerjaan dengan baik.
2 (Exceeded) → Melebihi ekspektasi dalam kinerja.
3 (Need Improvement) → Performa kurang baik, tapi belum sampai tahap PIP.
"""